# Load data

In [37]:
import scipy.io
saved  = scipy.io.loadmat("../matlab/utility.mat")
attacker_utility = saved["attacker_utility"]
defender_utility = saved["defender_utility"]

# Configure numpy display options

In [38]:
import numpy as np
import scipy.optimize

np.set_printoptions(
    suppress=True,
    linewidth=180
)

num_options = len(attacker_utility)

# Setup solver for defender

In [39]:
c = np.zeros(num_options+1)
c[0] = -1
print(c)

[-1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [40]:
A_ub = np.ones((num_options, num_options+1))
A_ub[:,1:] = defender_utility*-1
print(A_ub)

[[  1.       2.2041   7.11     7.11     7.11     2.2041   2.2041   2.2041   7.11     7.11     7.11   999.     999.     999.     999.     999.    ]
 [  1.      13.12     0.3936  13.12    13.12     0.3936  13.12    13.12     0.3936   0.3936  13.12   999.     999.     999.     999.     999.    ]
 [  1.       8.       8.       1.84     8.       8.       1.84     8.       1.84     8.       1.84   999.     999.     999.     999.     999.    ]
 [  1.       5.4      5.4      5.4      1.242    5.4      5.4      1.242    5.4      1.242    1.242  999.     999.     999.     999.     999.    ]
 [  1.      15.3241   7.5036  20.23    20.23     2.5977  15.3241  15.3241   7.5036   7.5036  20.23   999.     999.     999.     999.     999.    ]
 [  1.      10.2041  15.11     8.95    15.11    10.2041   4.0441  10.2041   8.95    15.11     8.95   999.     999.     999.     999.     999.    ]
 [  1.       7.6041  12.51    12.51     8.352    7.6041   7.6041   3.4461  12.51     8.352    8.352  999.     999.    

In [41]:
print(max(A_ub.flatten()))
print(min(A_ub.flatten()))

999.0
0.39360000000000034


In [42]:
# b = np.zeros((num_options,1))
b_ub = np.zeros(num_options)
print(b_ub)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [43]:
A_eq = np.ones((1, num_options+1))
A_eq[0][0] = 0
print(A_eq)

[[0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


In [44]:
b_eq = 1
print(b_eq)

1


In [45]:
lb = np.zeros(num_options+1)
lb[0] = -10000

ub = np.ones(num_options+1)
ub[0] = 10000

bounds = np.asarray([lb, ub]).transpose()
print(bounds)

[[-10000.  10000.]
 [     0.      1.]
 [     0.      1.]
 [     0.      1.]
 [     0.      1.]
 [     0.      1.]
 [     0.      1.]
 [     0.      1.]
 [     0.      1.]
 [     0.      1.]
 [     0.      1.]
 [     0.      1.]
 [     0.      1.]
 [     0.      1.]
 [     0.      1.]
 [     0.      1.]]


# Solve for defender

In [46]:
# zz = []
result = scipy.optimize.linprog(
    c=c,
    A_ub=A_ub,
    b_ub=b_ub,
    A_eq=A_eq,
    b_eq=b_eq,
    bounds=bounds,
    method="simplex",
    # method="highs",
    # options={"presolve":False},
    # callback = lambda x: zz.append(x.x)
)
if result.success:
    print(result.x)
else:
    print(result)
# todo: trim result to not include bad states? idk
# todo: don't evaluate for bad states, cut from matrix I guess
# this has -999 for first item because there are no states where defender doesn't have -999 in a row since the attacker can't make certain attacks

[-14.7436   0.       0.       0.       0.       0.       0.       0.       1.       0.       0.       0.       0.       0.       0.       0.    ]


# Evaluate defender results

In [47]:
# actual = defender_utility @ result.x[1:]
actual = defender_utility @ result.x[1:]
print(actual, sum(actual))

[ -7.11    -0.3936  -1.84    -5.4     -7.5036  -8.95   -12.51    -2.2336  -5.7936  -7.24    -9.3436 -12.9036 -14.35    -7.6336 -14.7436] -117.94880000000097


In [48]:
guess = defender_utility @ [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0]
print(guess, sum(guess))

[ -7.11    -0.3936  -8.      -5.4     -7.5036 -15.11   -12.51    -8.3936  -5.7936 -13.4    -15.5036 -12.9036 -20.51   -13.7936 -20.9036] -167.2288


# Setup solver for attacker

In [49]:
A_ub = np.ones((num_options, num_options+1))
A_ub[:,1:] = attacker_utility.transpose()*-1
print(A_ub)

[[  1.      -2.2041 -13.12    -8.      -5.4    -15.3241 -10.2041  -7.6041 -21.12   -18.52   -13.4    -23.3241 -20.7241 999.     999.     999.    ]
 [  1.      -7.11    -0.3936  -8.      -5.4     -7.5036 -15.11   -12.51    -8.3936  -5.7936 -13.4    -15.5036 -12.9036 999.     999.     999.    ]
 [  1.      -7.11   -13.12    -1.84    -5.4    -20.23    -8.95   -12.51   -14.96   -18.52    -7.24   -22.07   -25.63   999.     999.     999.    ]
 [  1.      -7.11   -13.12    -8.      -1.242  -20.23   -15.11    -8.352  -21.12   -14.362   -9.242  -28.23   -21.472  999.     999.     999.    ]
 [  1.      -2.2041  -0.3936  -8.      -5.4     -2.5977 -10.2041  -7.6041  -8.3936  -5.7936 -13.4    -10.5977  -7.9977 999.     999.     999.    ]
 [  1.      -2.2041 -13.12    -1.84    -5.4    -15.3241  -4.0441  -7.6041 -14.96   -18.52    -7.24   -17.1641 -20.7241 999.     999.     999.    ]
 [  1.      -2.2041 -13.12    -8.      -1.242  -15.3241 -10.2041  -3.4461 -21.12   -14.362   -9.242  -23.3241 -16.5661

# Solve for attacker

In [50]:
# zz = []
result = scipy.optimize.linprog(
    c=c,
    A_ub=A_ub,
    b_ub=b_ub,
    A_eq=A_eq,
    b_eq=b_eq,
    bounds=bounds,
    method="simplex",
    # method="highs",
    # options={"presolve":False},
    # callback = lambda x: zz.append(x.x)
)
if result.success:
    print(result.x)
else:
    print(result)
# todo: trim result to not include bad states? idk
# todo: don't evaluate for bad states, cut from matrix I guess
# this has -999 for first item because there are no states where defender doesn't have -999 in a row since the attacker can't make certain attacks

[4.4377 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     1.     0.     0.     0.     0.    ]


# Evaluate attacker results

In [51]:
# actual = defender_utility @ result.x[1:]
actual = attacker_utility.transpose() @ result.x[1:]
print(actual, sum(actual))

[23.3241 15.5036 22.07   28.23   10.5977 17.1641 23.3241  9.3436 15.5036 22.07    4.4377 10.5977 17.1641  9.3436  4.4377] 233.11159999999964


In [52]:
guess = attacker_utility.transpose() @ [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0]
print(guess, sum(guess))

[13.12    0.3936 13.12   13.12    0.3936 13.12   13.12    0.3936  0.3936 13.12    0.3936  0.3936 13.12    0.3936  0.3936] 94.98880000000004
